In [ ]:
import os.path, random, shutil, tweepy, re, nltk, glob
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize as wt
from autocorrect import spell
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_validate, cross_val_predict
from numba import jit, cuda 

In [ ]:
def cros(Knn,X,Y,name):
    VAL = cross_validate(knn, X, Y, cv=10, return_train_score=True, scoring=['f1','precision','recall'])
    save_path = 'C:/Users/higor.silva/TCC/Resultados/Cros/'
    name_of_file = name
    completeName = os.path.join(save_path, name_of_file+".txt")
    file1 = open(completeName, "w")
    file1.write(str(VAL))
    file1.close()

In [ ]:
def aval_model_macro(y_test,pred,name):
    avl = precision_recall_fscore_support ( y_test,pred, average = 'macro' )
    save_path = 'C:/Users/higor.silva/TCC/Resultados/Macro/'
    name_of_file = name
    completeName = os.path.join(save_path, name_of_file+".txt")
    file1 = open(completeName, "w")
    file1.write(str(avl))
    file1.close()    

In [ ]:
def aval_model_micro(y_test,pred,name):
    avl2 = precision_recall_fscore_support ( y_test,pred, average = 'micro' )
    save_path = 'C:/Users/higor.silva/TCC/Resultados/Micro/'
    name_of_file = name
    completeName = os.path.join(save_path, name_of_file+".txt")
    file1 = open(completeName, "w")
    file1.write(str(avl2))
    file1.close()    

In [ ]:
K=3
while K <=17: 
    for j in range(30):
        
        files = glob.glob('C:/Users/higor.silva/TCC/base/true/*')
        for f in files:
            os.remove(f)
        
        source= "C:/Users/higor.silva/TCC/base/txtTribuna"
        dest= "C:/Users/higor.silva/TCC/base/true"
        no_of_files=2980

        #print("%"*25+"{ Details Of Transfer }"+"%"*25)
        #print("\n\nList of Files Moved to %s :-"%(dest))

        for i in range(no_of_files):    
            random_file=random.choice(os.listdir(source))
            #print("%d} %s"%(i+1,random_file))
            source_file="%s\%s"%(source,random_file)
            dest_file=dest    
            shutil.copy(source_file,dest_file)

        #print("\n\n"+"$"*33+"[ Files Moved Successfully ]"+"$"*33)
        
        pasta = 'base'
        labels = {'true': 1, 'fake': 0}
        df = pd.DataFrame()
        
        for f in ('true', 'fake'):
            path = os.path.join(pasta,f)
            for file in os.listdir (path) :
                with open(os.path.join(path, file),'r', encoding='ISO-8859-1') as infile:
                    txt = infile.read()
                df = df.append([[txt, labels[f]]],ignore_index=True)
        df.columns = ['noticia', 'rotulo']
        df.to_csv('noticias_data.csv', index=False, encoding='utf-8' )
        
        auth = tweepy.OAuthHandler('XXXXXXXXXXXXXXXXXX', 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
        auth.set_access_token('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
                'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
        api = tweepy.API(auth)
        

        def obter_tweets(usuario, limite=10):
            resultados = api.user_timeline(screen_name=usuario, count=limite, tweet_mode='extended')
            tweets = [] 
            for r in resultados:
                tweet = re.sub(r'http\S+', '', r.full_text)
                tweets.append(tweet.replace('\n', ' ')) 
            return tweets 
        
        tweets2 = obter_tweets(usuario='@GuilhermeBoulos', limite=15)
        df2 = pd.DataFrame(tweets2)
        df2.columns = ['noticia']
        frames = pd.concat([df, df2],ignore_index=True, sort = False)
        frames = frames.replace(np.nan, 0)
        #frames
        
        stemmer = nltk.stem.RSLPStemmer()
        arq_stopwords = open("ListaStopWords.txt", "r", encoding='utf-8')
        stopwords = arq_stopwords.read()
        #print(stopwords)
        
        
        def aplicastemmer(texto):    
            frases_processed = []    
            for palavra in texto:
                comstemming = [str(stemmer.stem(p)) for p in palavra.split() if p not in stopwords]
                frases_processed.append(comstemming)            
            return frases_processed
        
        noticiario = frames['noticia'].str.lower()
        noticia = aplicastemmer(noticiario)
        
        matrix = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

        X = matrix.fit_transform(noticia).toarray()
        Y = frames['rotulo']
        
        knn = KNeighborsClassifier(metric = 'cosine', n_neighbors=K)
        x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)
        
        knn.fit(x_train,y_train)      
        pred = knn.predict(x_test)
        
        name = 'K'+str(K)+'J'+str(j)
        
        aval_model_micro(y_test,pred,str(name))
        aval_model_macro(y_test,pred,str(name))
        cros(knn,X,Y,str(name))
        
    K = K+2

In [ ]:
#res = np.array(resultados)

In [ ]:
#print(resultados_macro)

In [ ]:
#print(resultados_micro)

In [ ]:
#print(cros)

In [ ]:
#res = np.delete(res, 3, 1)

In [ ]:
#res

In [ ]:
#soma = np.sum(res, axis=0)